# MD-WERP Strategic Project 11.3

Requirements:

Need a Google Earth Engine login, subject to their Terms and Conditions

In [ ]:
# !pip install earthengine-api --upgrade

## Import modules

In [ ]:
import ee
import folium
import geopandas as gpd
import time
from time import gmtime, strftime, localtime

## Connect to Google Earth Engine
You will need your own sign-in

In [ ]:
ee.Authenticate()
ee.Initialize()

## Define user variables

Import datasets

In [ ]:
catchments = ee.FeatureCollection('users/Projects/Hunt/AHGFContractedCatchemnts')
anae = ee.Image('users/Projects/anae3_ls_max_area_0')
mdb = ee.FeatureCollection('users/Projects/MDB_boundary_2km')
nb = ee.FeatureCollection('users/Projects/NB_catchments')
sb = ee.FeatureCollection('users/Projects/SB_catchments') #https://code.earthengine.google.com/?asset=users/Projects/SB_catchments

Set user defined variables

In [ ]:
# Test mode
test_mode = 'no' # yes to not submit tasks, no to submit tasks

# Set parameters for Image Collections
# Dates can currently only be intervals of years
start_date = '1997-01-01' # 1984-01-01 landsat 5 archive begins
end_date = '2023-01-01'
cloud_percent = 20 # Max percentage allowed
slc_data = 'no' # yes for include data, no for throw affected data away
clip_edges = 'yes' # yes to clip by 6km for l5 and l7, no to not

# Set parameters for aggregations
timeframe = ee.String('month') 
advance_overlap = 1

# Set export variables
crs = 'EPSG:32655'
crsTransform = [30,0,309885,0,-30,-3881085]
epsg = '32655'
bucket = 'sharebucketgee'
scale = 30
fileNamePrefix = 'MDWERP11/Project_113_January_2023/'

# Boundaries
bounds = mdb.geometry()
bounds_list = [sb.geometry(),nb.geometry()]

## Define functions

Unmask ANAE so that '0' is all the 'non-ANAE' classified area

In [ ]:
zeroIm = ee.Image(0).rename('b1').toShort()
anae1 = ee.ImageCollection([zeroIm,anae.unmask()]).mosaic()

In [ ]:
# Define a function to scale the data and mask unwanted pixels.
def maskL457sr(image):
    '''
    maskL457sr from https://developers.google.com/earth-engine/guides/ic_visualization
    '''
    # Bit 0 - Fill
    # Bit 1 - Dilated Cloud
    # Bit 2 - Unused
    # Bit 3 - Cloud
    # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)

    # Apply the scaling factors to the appropriate bands.
    # opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    # thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True)\
        .addBands(thermalBands, None, True)\
        .updateMask(qaMask)\
        .updateMask(saturationMask)


# Landsat (4), 5 and 7
def renameBands57(img):
    return img.select(['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7','QA_RADSAT','QA_PIXEL'])\
              .rename(['blue','green','red','nir','swir1','swir2','QA_RADSAT','QA_PIXEL'])

# Landsat 8
def renameBands89(img):
    return img.select(['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7','QA_RADSAT','QA_PIXEL'])\
              .rename(['blue','green','red','nir','swir1','swir2','QA_RADSAT','QA_PIXEL'])

# Function to clip edges of Landsat
def clipEdges(img):
    return img.clip(img.geometry().buffer(-6000))

# Function to clip edges of Landsat
def clipBasin(img):
    return img.clip(bounds)

# Function to add MNDWI
def addMNDWI(image):
    b1 = image.select('green')
    b2 = image.select('swir1')
    new_band = b1.subtract(b2).divide(b1.add(b2)).rename('mndwi')
    return image.addBands(new_band)

# Function to keep only water
def onlyW(image):
    water = image.select('mndwi').gt(0).rename('water')
    return image.addBands(water).selfMask()

# Funciton to mask bad pixels
def maskBad(image):
    bad_mask = image.select(['blue','green','red','nir','swir1','swir2'])\
                    .reduce(ee.Reducer.min())\
                    .gt(0)
    return image.mask(bad_mask)

Define functions - to apply after monthly mosaic

In [ ]:
# Function to calculate area of bad pixels from image - Landsat
# Only works in footprint of Image
def getBadPixel(image):
    """
    Returns an image where 1 = bad quality pixels, and good quality pixels are masked
    """
    constant = ee.Image(0).multiply(image.select('QA_PIXEL_max'))\
                          .add(ee.Image(1))\
                          .unmask()\
                          .eq(0)\
                          .selfMask()\
                          .rename('bad_pixel')
    return image.addBands(constant)

# Function to calculate bad pixel area
def getBadPixelArea(image):
    stats = image.select('bad_pixel').eq(1).multiply(ee.Image.pixelArea()).rename('bad_pixel_area')
    return image.addBands(stats)

# Function to calculate water area
def getWaterArea(image):
    stats = image.select('water_max').eq(1).multiply(ee.Image.pixelArea()).rename('water_area')
    return image.addBands(stats)


# Function to calculate clear pixel
def getClear(image):
    stats = ee.Image(1).mask(image.select('bad_pixel').unmask()\
                                  .add(image.select('water_max').unmask())\
                                  .eq(0))\
                        .rename('clear')
    return image.addBands(stats)

# Function to calculate clear pixel area
def getClearArea(image):
    stats = image.select('clear').multiply(ee.Image.pixelArea()).rename('clear_area')
    return image.addBands(stats)

Make functions to add band name as a property

In [ ]:
def addName_water(element):
    ndict = {'band_name': 'water_area'};
    nowhereFeature = ee.Feature(None, ndict);
    return element.copyProperties(nowhereFeature,['band_name']);

def addName_clear(element):
    ndict = {'band_name': 'clear_area'};
    nowhereFeature = ee.Feature(None, ndict);
    return element.copyProperties(nowhereFeature,['band_name']);

def addName_bad(element):
    ndict = {'band_name': 'bad_area'};
    nowhereFeature = ee.Feature(None, ndict);
    return element.copyProperties(nowhereFeature,['band_name']);

Make function to flatten groups

In [ ]:
# Convert output column list to columns
def formatGroups(feature):
    list1 = ee.List(ee.Feature(feature).get('groups'))
    keys = list1.map(lambda o: ee.Number(ee.Dictionary(o).get('group')).format('%d'))
    values = list1.map(lambda o: ee.Dictionary(o).get('sum'))
    return ee.Feature(feature.geometry(), ee.Dictionary.fromLists(keys, values))\
             .copyProperties(feature,['UID','date','band_name'])

Create a dummy row that contains all the values in the ANAE. This is the template used in the export, which converts the json into a csv. 

In [ ]:
anae_values = list(range(0,71))
anae_strings = [str(x) for x in anae_values]
fields = anae_strings + ['UID','band_name','date']
field_dict = ee.FeatureCollection([ee.Feature(None, ee.Dictionary.fromLists(fields, fields))])

## Process the images

Import Image Collections

In [ ]:
# Import collections
l9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")

Process image collections

In [ ]:
start_year = int(start_date.split('-')[0])
end_year = int(end_date.split('-')[0])
length_list = list(range(start_year,end_year))


# List to append task
id_list = []

for year in length_list:
    print(year,year+1)
    start = ee.Date.fromYMD(year,1,1)
    end = ee.Date.fromYMD(year+1,1,1)
    print(start.getInfo(),end.getInfo())

    # Do collection specific mapping
    l5c = l5.map(maskL457sr).map(renameBands57)
    if slc_data == 'yes':
        l7c = l7.map(maskL457sr).map(renameBands57)
    elif slc_data == 'no':
        l7c = l7.map(maskL457sr).map(renameBands57).filterDate('1999-01-01','2003-05-31') # Remove SLC error affected images
    l8c = l8.map(maskL457sr).map(renameBands89)
    l9c = l9.map(maskL457sr).map(renameBands89)
    if clip_edges == 'yes':
        col = (l5c.map(clipEdges)).merge(l7c.map(clipEdges)).merge(l8c).merge(l9c)\
                 .filterDate(start,end)\
                 .filterBounds(bounds)\
                 .filter(ee.Filter.lt('CLOUD_COVER',cloud_percent))\
                 .map(addMNDWI)\
                 .map(onlyW)\
                 .map(maskBad)
    elif clip_edges == 'no':
        col = l5c.merge(l7c).merge(l8c).merge(l9c)\
                 .filterDate(start,end)\
                 .filterBounds(bounds)\
                 .filter(ee.Filter.lt('CLOUD_COVER',cloud_percent))\
                 .map(onlyW)\
                 .map(addMNDWI)\
                 .map(maskBad)

    ####################################################################################################################

    # Process to make monthly mosaics 
    months = end.difference(start, timeframe)
    datelist = ee.List.sequence(0, months.int()).map(lambda count: start.advance(count, timeframe))

    # This function creates a list of images
    # This finds the max of every band
    def makeDatelist(d):
        begin = ee.Date(d)
        end = begin.advance(advance_overlap, timeframe)
        imgcol = col.filterDate(begin, end)
        monthsum= imgcol.reduce(ee.Reducer.max())\
                        .set({'date':begin}) # Set date
        return monthsum

    dummy = datelist.map(makeDatelist)

    # Turn list into image collection
    col = ee.ImageCollection(dummy)

    # Get rid of empty images
    c = col.map(lambda img: img.set('count', img.bandNames().length()))\
           .filter(ee.Filter.gt('count', 0))\
           .map(getBadPixel)\
           .map(getBadPixelArea)\
           .map(getWaterArea)\
           .map(getClear)\
           .map(getClearArea)\
           .map(clipBasin)

    ####################################################################################################################

    # Create list to iterate through
    length_list = list(range(0,end.difference(start,'month').round().getInfo())) # Max possible images

    for i in length_list:
        date = ee.Date(datelist.get(i)).format('YYYY-MM-dd').getInfo()
        print(date)

        try:
            stat_im = c.filter(ee.Filter.eq('date',ee.Date(date))).first()

            description = 'Stat_im_'+'slc_'+str(slc_data)+'_clip57_'+str(clip_edges) + '_cplt_' +str(cloud_percent)
            print(description)
            task = ee.batch.Export.image.toAsset(image=stat_im.select('clear_area','water_area','bad_pixel_area'), 
                                                 description= description + '_' + date, 
                                                 assetId='projects/rock-hangar-240123/assets/MDWERP_113_January_2023/'+ description + '_' + ee.Date(datelist.get(i)).format('YYYYMMdd').getInfo(), 
                                                 region=mdb.geometry(), 
                                                 scale=scale, 
                                                 crs=crs, 
                                                 crsTransform=crsTransform, 
                                                 maxPixels=100000000000, 
                                                    )
            print(task)
            if test_mode == 'no':
                task.start()
                list_id = task.status()['name']
                id_list.append(list_id)
            elif test_mode == 'yes':
                pass
        
        except Exception as e:
            print('Problem with the export')
            print(e)  

    # Wait 3hours for each 12 months to save out
    print('Started: ',str(year),' at ',strftime("%Y-%m-%d %H:%M:%S", localtime()))
    
    if test_mode == 'no':
        time.sleep(1*10800) 
    elif test_mode == 'yes':
        pass
    

# Calculate the areas

In [ ]:
# List assets
asset_dir = f"projects/XXXXXX/assets/MDWERP_113_January_2023/"
asset_list = ee.data.listAssets({'parent':asset_dir})['assets']

In [ ]:
for i in bounds_list:
    if i == sb.geometry():
        prefix = 'southb_'
        export_areas = sb
    elif i == nb.geometry():
        prefix = 'northb_'
        export_areas = nb

    test_mode = 'no'

    for asset in asset_list:
        
        date = asset['name'][-8:]

        try:
            stat_im = ee.Image(asset['name'])
            print(asset['name'])

            masked_stat_im = stat_im.unmask().addBands(anae1.rename('anae_value'))

            # Water
            sum_table_water = masked_stat_im.select(['water_area','anae_value'])\
                                   .reduceRegions(collection=export_areas,
                                                  reducer= ee.Reducer.sum().group(groupField=1),
                                                  crs= crs,
                                                  crsTransform= crsTransform).map(lambda feature: feature.set({'date':date}))\
                                                                             .map(addName_water)

            # Clear
            sum_table_clear = masked_stat_im.select(['clear_area','anae_value'])\
                                   .reduceRegions(collection=export_areas,
                                                  reducer= ee.Reducer.sum().group(groupField=1),
                                                  crs= crs,
                                                  crsTransform= crsTransform).map(lambda feature: feature.set({'date':date}))\
                                                                             .map(addName_clear)
            # Bad
            sum_table_bad = masked_stat_im.select(['bad_pixel_area','anae_value'])\
                                   .reduceRegions(collection=export_areas,
                                                  reducer= ee.Reducer.sum().group(groupField=1),
                                                  crs= crs,
                                                  crsTransform= crsTransform).map(lambda feature: feature.set({'date':date}))\
                                                                             .map(addName_bad)


            # Combine all 3 band results
            bin_area = ee.FeatureCollection(sum_table_water.toList(sum_table_water.size())\
                                    .cat(sum_table_clear.toList(sum_table_clear.size()))\
                                    .cat(sum_table_bad.toList(sum_table_bad.size())));

            # Flatten groups
            flattened = bin_area.map(formatGroups).select([".*"], None, False)

            # Add row header dummy
            result = field_dict.merge(flattened)

            description= prefix + asset['name'].split('/')[4][8:]
            print('Will try exporting: ',description,' at ',strftime("%Y-%m-%d %H:%M:%S", localtime()))

            try:
                task = ee.batch.Export.table.toCloudStorage(collection= result,
                                                            description= description, 
                                                            bucket= bucket,
                                                            fileNamePrefix = fileNamePrefix + description)
                if test_mode == 'yes':
                    pass
                elif test_mode == 'no':
                    task.start()

                print('Exporting: ',description,' at ',strftime("%Y-%m-%d %H:%M:%S", localtime()))
                time.sleep(1*60)  

            except Exception as e:
                print('Problem with the export')
                print(e)  
        except Exception as e:
            print()
            print(e)    

Check progress in the [Task Manager](https://code.earthengine.google.com/tasks)

In [ ]:
# # Kill all tasks in emergency
# for task in ee.batch.Task.list():
#     task.cancel()

## Calculate Area for catchments using a blank image

In [ ]:
# Create blank image
blank = ee.Image(1)

# Calculate area
catch_table = blank.multiply(ee.Image.pixelArea())\
                   .reduceRegions(collection=catchments,
                                  reducer= ee.Reducer.sum(),
                                  crs= crs,
                                  crsTransform= crsTransform)

# Export
task = ee.batch.Export.table.toCloudStorage(collection= catch_table.select(['sum','UID'], None, False),
                                            description= 'Area_of_all_catchments', 
                                            bucket= bucket,
                                            fileNamePrefix = 'Project_113/'+'Area_of_all_catchments')
task.start()